In [20]:
import pandas as pd 

## 만든 QA 질문-답변정답 데이터셋 불러오기

In [21]:
qa_data = pd.read_excel('./experiment_dataset/qa_data_india.xlsx')

qa_data.head()

,question,ground_truth
0,"Flipkart는 언제 설립되었으며, 본사는 어디에 위치해 있나요?","Flipkart는 2007년에 설립되었으며, 본사는 벵갈루루에 위치해 있습니다."
1,인도 재무부의 Notification No. 09/2020-Customs (N.T....,2020년 2월 2일에 발표되었습니다.
2,카자흐스탄 내에서 한국 차량 브랜드의 시장 점유율은 얼마나 되나요?,카자흐스탄 내 신차 판매에서 한국 차량 브랜드 비중이 약 40%로 1위를 기록하고 ...
3,한전KPS가 파키스탄에서 운영 대금을 회수하지 못한 이유는 무엇인가요?,한전KPS는 파키스탄 중앙은행(SBP)이 외화 부족 상황을 이유로 운영 대금 해외송...
4,최근 인도의 수입규제 급증 배경은 무엇인가요?,최근 인도의 수입규제 급증은 인도의 국내 산업보호를 위한 정치적 요구와 당국의 통상...


In [22]:
print(len(qa_data))

10


### 실험을 위한 RAG chain 

In [23]:
from utils.RAGPipeLine import Ragpipeline
pipeline = Ragpipeline()
VectorDB = pipeline.init_vectorDB()

[초기화] vector_store 초기화 완료
[초기화] retriever 초기화 완료
[초기화] RAG chain 초기화 완료
[초기화] RAG title chain 초기화 완료
[초기화] RAG post chain 초기화 완료
[초기화] vector_store 초기화 완료


In [24]:
retriever1 = VectorDB.as_retriever(
            search_kwargs = {"k": 3},
            search_type   = "similarity"
        )

retriever2 = VectorDB.as_retriever(
            search_kwargs = {"k": 3},
            search_type   = "mmr"
        )

# basic_retriever = pipeline.init_retriever()
basic_retriever = VectorDB.as_retriever(
            search_kwargs = {"score_threshold": 0.5, "k": 3},
            search_type   = "similarity_score_threshold"
        )

mq_retriever = pipeline.init_multi_query_retriever()
sq_retriever = pipeline.init_self_query_retriever()
pd_retriever  = pipeline.init_parent_document_retriever()
web_retriever = pipeline.init_web_research_retriever()

In [25]:
query = '인도 통관 및 운송'

docs = sq_retriever.get_relevant_documents(query)

docs

```json
{
    "query": "인도 통관 및 운송",
    "filter": "NO_FILTER"
}
```

[Document(page_content='제Ⅲ절\n인도의 통관 절차\n01\n통관 조직 및 절차\n', metadata={'category': '3. 정책 및 무역', 'datetimes': '2024-07-08 16:42:32', 'filename': 'data/PDF_with_contents/3. 정책 및 무역/[정책][관세청][2019.11.07]신남방국 통관·통상환경 및 FTA 활용방안.pdf', 'page_no': 9, 'subcategory': '1. 통관 조직 및 절차'}),
 Document(page_content='인도 전문 식품수입 벤더현황\n다. 통관 및 검역 유의사항\n(1) 인도 통관, 사전 준비가 관건\n▪\n인도의 통관은 수입 통관 및 검역을 위한 사전 준비 절차가 중요함\n▪\n통관에 필요한 서류를 제출하면 서류의 기재 오류나 서류 상호간 불일치를\n수정하기가 매우 어려우며 이로 인해 통관 절차가 2~3주씩 지연되는 경우도\n있어 철저한 서류 작성이 필수적임\n▪\n또한 인도로 수출하는 물품의 경우 포장명세서(P/L)와 상업송장(C/I)을\n컨테이너별로 철저히 작성해야 함. 인도 세관은 한 컨테이너 안에서 박스별로\n화물의 일치 여부를 조사하는 등 실물 검사가 철저하며 서류와 화물 불일치\n시 통관이 불가능함\n▪\n특히 인도는 도로와 항만 등 인프라가 낙후되어 있고 운송수단도 노후하기\n때문에 내륙운송 과정에서 사고가 자주 발생하므로 내륙운송보험에 가입해야\n함\n43\n', metadata={'category': '3. 정책 및 무역', 'datetimes': '2024-07-08 16:41:51', 'filename': 'data/PDF_with_contents/3. 정책 및 무역/[정책][KATI 농식품수출정보][2019]인도 전문 식품수입.pdf', 'page_no': 21, 'subcategory': '다. 통관 및 검역 유의사항'}),
 Document(page_content='5 통관 및 운송\n가 통관제도\n통관 

In [26]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from utils.config import config
our_llm = ChatOpenAI(
            model       = "gpt-4o", # config['llm_predictor']['model_name'],
            temperature = config['llm_predictor']['temperature'],

        )

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from utils.prompt import qa_prompt2    

def set_chain(llm, retriever, prompt):
    
    question_answer_chain = create_stuff_documents_chain(llm, prompt)           # 문서 목록을 가져와서 모두 프롬프트로 포맷한 다음 해당 프롬프트를 LLM에 전달합니다.
    
    rag_chat_chain = create_retrieval_chain(retriever, question_answer_chain)  # 사용자 문의를 받아 리트리버로 전달하여 관련 문서를 가져옵니다. 그런 다음 해당 문서(및 원본 입력)는 LLM으로 전달되어 응답을 생성

    return rag_chat_chain


def start_experiment_and_save(qa_data, llm, retriever, prompt):
    rag_chain = set_chain(llm, retriever, prompt)
    
    qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke({'input':x})['answer'])
    qa_data['contexts'] = qa_data['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)])
    # 5분 걸림
    return qa_data

In [28]:
# setting1 
retriever1 = VectorDB.as_retriever(
            search_kwargs = {"k": 3},
            search_type   = "similarity"
        )

results = start_experiment_and_save(qa_data, our_llm, retriever1, qa_prompt2)

In [29]:
results.head()

,question,ground_truth,answer,contexts
0,"Flipkart는 언제 설립되었으며, 본사는 어디에 위치해 있나요?","Flipkart는 2007년에 설립되었으며, 본사는 벵갈루루에 위치해 있습니다.","Flipkart는 2007년 10월에 설립되었으며, 본사는 인도 벵갈루루에 위치해 ...",[Flipkart\n사이트 주소 httpswwwflipkartcom\n플립카트Fli...
1,인도 재무부의 Notification No. 09/2020-Customs (N.T....,2020년 2월 2일에 발표되었습니다.,인도 재무부의 Notification No. 09/2020-Customs (N.T....,[08\n2 인도의 무역구제규정 개정\n인도 재무부는 2020년 2월 2일자로 반덤...
2,카자흐스탄 내에서 한국 차량 브랜드의 시장 점유율은 얼마나 되나요?,카자흐스탄 내 신차 판매에서 한국 차량 브랜드 비중이 약 40%로 1위를 기록하고 ...,제공된 자료에는 카자흐스탄 내에서 한국 차량 브랜드의 시장 점유율에 대한 정보가 포...,[차이즈의 성공률을 높일 수 있을 것으로 예상되기 때문이다\n2 빠른 시승 및 유지...
3,한전KPS가 파키스탄에서 운영 대금을 회수하지 못한 이유는 무엇인가요?,한전KPS는 파키스탄 중앙은행(SBP)이 외화 부족 상황을 이유로 운영 대금 해외송...,제공된 자료에는 한전KPS가 파키스탄에서 운영 대금을 회수하지 못한 구체적인 이유에...,[여기서 발생한 비용을 어디까지 포함시킬 수 있을 것이냐는 얼마든지 논의 가능한\n...
4,최근 인도의 수입규제 급증 배경은 무엇인가요?,최근 인도의 수입규제 급증은 인도의 국내 산업보호를 위한 정치적 요구와 당국의 통상...,최근 인도의 수입규제 급증 배경은 여러 가지 요인에 기인합니다. 주요 배경은 다음과...,[상반기 수출은 2078억 달러로 전년 동기 대비 127 감소 수입은 3202억 달...


In [31]:
results.to_excel('./experiment_dataset/QA/setting1_retriever1_003.xlsx', index=False)

### 반복

In [32]:
for i in [3,5,7,10]:
    # setting1 
    retriever1 = VectorDB.as_retriever(
                search_kwargs = {"k": i},
                search_type   = "similarity"
            )

    results = start_experiment_and_save(qa_data, our_llm, retriever1, qa_prompt2)
    results.to_excel('./experiment_dataset/QA/setting1_retriever1_{:03d}.xlsx'.format(i), index=False)
    print(i)

3
5
7
10


### 2.retriever2 실험

In [33]:
for i in [3,5,7,10]:
    # setting2
    retriever2 = VectorDB.as_retriever(
            search_kwargs = {"k": i},
            search_type   = "mmr"
        )

    results = start_experiment_and_save(qa_data, our_llm, retriever2, qa_prompt2)
    results.to_excel('./experiment_dataset/QA/setting2_retriever2_{:03d}.xlsx'.format(i), index=False)
    print(i)

3
5
7
10


### 3. mmr - lambda_mult

In [34]:
# Retrieve more documents with higher diversity
# Useful if your dataset has many similar documents

for i in [3,5,7,10]:
    for j in range(1,8,1):
        # setting2
        retriever = VectorDB.as_retriever(
                search_kwargs = {'k': i, 'lambda_mult': j/10},
                search_type   = "mmr"
            )

        results = start_experiment_and_save(qa_data, our_llm, retriever, qa_prompt2)
        results.to_excel('./experiment_dataset/QA/setting3_retriever3_{:03d}_{:03d}.xlsx'.format(i,j), index=False)
        print(i, j)

3 1
3 2
3 3
3 4
3 5
3 6
3 7
5 1
5 2
5 3
5 4
5 5
5 6
5 7
7 1
7 2
7 3
7 4
7 5
7 6
7 7
10 1
10 2
10 3
10 4
10 5
10 6
10 7


### 4. our basic_retriever 실험

In [35]:
# Only retrieve documents that have a relevance score
# Above a certain threshold

for i in [3,5,7,10]: 
    for j in range(1,8,1): 
        # setting3 
        our_retriever = VectorDB.as_retriever(
                search_kwargs = {"score_threshold": j/10, "k": i},
                search_type   = "similarity_score_threshold"
            )

        results = start_experiment_and_save(qa_data, our_llm, our_retriever, qa_prompt2)
        results.to_excel('./experiment_dataset/QA/setting4_our_retriever_{:03d}_{:03d}.xlsx'.format(i,j), index=False)
        print(i, j)

3 1
3 2
3 3
3 4
3 5
3 6
3 7
5 1
5 2
5 3
5 4
5 5
5 6
5 7
7 1
7 2
7 3
7 4
7 5
7 6
7 7
10 1
10 2
10 3
10 4
10 5
10 6
10 7


### 5. multi_query_retriever

In [36]:
# multi_query_retriever는 딱히 인자가 없다.

results = start_experiment_and_save(qa_data, our_llm, mq_retriever, qa_prompt2)
results.to_excel('./experiment_dataset/QA/setting5_mq_retriever.xlsx', index=False)


1. 언제 Flipkart가 설립되었고, 본사는 어디에 위치하고 있나요?
2. Flipkart 설립일은 언제이며, 본사는 어디에 위치하고 있습니까?
3. Flipkart 설립 날짜와 본사 위치에 대해 알려주세요.1. 인도 재무부의 Notification No. 09/2020-Customs (N.T.)는 언제 공표되었습니까?
2. 인도 재무부의 공고 번호 09/2020-Customs (N.T.)는 언제 발표되었나요?
3. 인도 재무부의 공고 번호 09/2020-Customs (N.T.) 발표일은 언제인가요?1. 한국 자동차 브랜드가 카자흐스탄 시장에서 차지하는 점유율은 어느 정도인가요?
2. 카자흐스탄 내에서 한국 자동차 브랜드의 시장 점유율은 어떻게 되나요?
3. 한국 자동차 브랜드가 카자흐스탄에서 얼마나 시장을 점유하고 있나요?1. 한전KPS가 파키스탄에서 운영 대금을 회수하지 못한 원인은 무엇인가요?
2. 파키스탄에서 한전KPS가 운영 대금을 회수하지 못한 이유는 무엇인가요?
3. 한전KPS가 파키스탄에서 운영 대금을 회수하지 못한 이유에 대해 알고 계신가요?1. 인도의 수입규제 급증 배경은 무엇인가요?
2. 최근 인도에서 수입규제가 왜 급증했나요?
3. 인도의 수입규제가 급증한 이유는 무엇인가요?1. 리비아 당국이 주문하는 인프라 구축 사업에 대해 어떤 측면을 주의깊게 살펴봐야 할까요?
2. 리비아 당국이 발주하는 인프라 구축 사업을 면밀히 검토할 때 고려해야 할 사항은 무엇인가요?
3. 인프라 구축 사업에 대한 리비아 당국의 발주를 검토할 때 주목해야 할 중요한 측면은 무엇인가요?1. 어떤 위험이 원산지 검증 시 발생할 수 있는지 알려주세요.
2. 원산지 검증 과정에서 발생할 수 있는 위험에는 무엇이 있을까요?
3. 원산지를 확인하는 과정에서 발생할 수 있는 위험 사항에 대해 설명해주세요.What was the export amount of South Korea in 2018?
How much did South Korea export in 2018?
C

### 6. self_query_retriever

In [37]:
# self_query_retriever는 딱히 인자가 없다.

results = start_experiment_and_save(qa_data, our_llm, sq_retriever, qa_prompt2)
results.to_excel('./experiment_dataset/QA/setting6_sq_retriever.xlsx', index=False)


```json
{
    "query": "Flipkart establishment headquarters location",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "Notification No. 09/2020-Customs (N.T.)",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "한국 차량 브랜드의 시장 점유율",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "한전KPS 파키스탄 운영 대금 회수 이유",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "인도의 수입규제",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "인프라 구축 사업",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "위험",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "2018년 한국의 수출액",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "한-인도 CEPA를 적용하여 수출하기 위해 필요한 서류",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "인도 내 수입업자의 이름 및 주소는 필수 요구사항인가요?",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "Flipkart establishment headquarters location",
    "filter": "NO_FILTER"
}
``````json
{
    "query": "Notification No. 09/2020-Customs (N.T.)",
    "filter": "NO_FILTER"
}
``````jso

### 7. parent_document_retriever

를 사용할 수 없다.

In [38]:
# # self_query_retriever는 딱히 인자가 없다.

# results = start_experiment_and_save(qa_data, our_llm, pd_retriever, qa_prompt2)
# results.to_excel('./experiment_dataset/QA/setting7_pd_retriever.xlsx', index=False)


In [39]:
results.tail()

,question,ground_truth,answer,contexts
5,리비아 당국이 발주하는 인프라 구축 사업에 대해 어떤 점을 면밀히 검토해야 하나요?,"리비아 당국이 발주하는 각종 인프라 구축 내지 복구 사업에 있어 실질적 이행 능력,...",리비아 당국이 발주하는 인프라 구축 사업에 참여하려면 다음과 같은 여러 가지 중요한...,[인프라 조성 비용의 25 지원 최대 1천만 루피\n구축 지원\n1326 안드라프라...
6,원산지 검증 시 어떤 위험이 있을 수 있나요?,"상대국 HS코드로만 원산지결정기준이 충족되는 경우, 원산지 검증 시 불충족 위험이 ...",원산지 검증 시 여러 가지 위험이 있을 수 있습니다. 다음은 주요 위험 요소들입니다...,[시로 발생하여 뉴스 이슈화된 바 있다\n 따라서 성범죄 예방을 위해 여성의 경우 ...
7,2018년 한국의 수출액은 얼마였나요?,2018년 한국의 수출액은 6천억 달러였습니다.,제공된 자료에는 2018년 한국의 수출액에 대한 정보가 포함되어 있지 않습니다. 한...,[국제관광 출발 횟수100만 2038 2187 2394 263 2692\n국제관광 ...
8,한-인도 CEPA를 적용하여 수출하기 위해 필요한 서류는 무엇인가요?,한-인도 CEPA를 적용하여 수출하기 위해서는 원산지기준을 충족한다는 것을 증명하는...,한-인도 CEPA(Comprehensive Economic Partnership A...,[제Ⅱ절\n한-인도 CEPA 활용방안\n02\n한-인도 CEPA\n수출유망품목 및\...
9,인도 내 수입업자의 이름 및 주소는 필수 요구사항인가요?,국내 제조된 제품의 경우 인도 내 수입업자의 이름 및 주소는 생략 가능합니다.,제공된 자료에는 인도 내 수입업자의 이름 및 주소가 필수 요구사항인지에 대한 정보가...,[PostPaid후불결제의 경우 상기 서류 외에도 거주증명서와 같은 별도의 증빙서류...
